In [1]:
# dependencies
from sqlalchemy import create_engine, inspect, MetaData, select, text, Table, func
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

# Create an engine that can talk to the database
engine = create_engine("sqlite:///Resources/la_crime.db", connect_args={'timeout': 30}, echo=False)
conn = engine.connect()

## Checking Database Data - using SQL

In [2]:
# Query data from crime table
crime_data = conn.execute("SELECT * FROM crime")

i = 0
for record in crime_data:
    print(record)
    i += 1
    if i == 10:
        break

(10304468, 2020, 1, 'January', '31-40', 'Black', 'Female', 34.0141, -118.2978, 3, 624, 501)
(20305364, 2019, 1, 'January', '41-50', 'Hispanic', 'Male', 34.0055, -118.2915, 3, 626, 502)
(170701073, 2018, 1, 'January', '11-20', 'Black', 'Male', 34.0421, -118.3456, 7, 888, 501)
(180100001, 2018, 9, 'September', '31-40', 'White', 'Female', 34.0382, -118.2889, 20, 510, 101)
(180100513, 2018, 1, 'January', '51-60', 'Black', 'Male', 34.0328, -118.2647, 1, 624, 101)
(180100514, 2018, 1, 'January', '31-40', 'White', 'Female', 34.0428, -118.2532, 1, 230, 108)
(180100516, 2018, 1, 'January', '11-20', 'Other', 'Male', 34.0454, -118.2422, 1, 740, 101)
(180100525, 2018, 1, 'January', '41-50', 'Hispanic', 'Male', 34.0487, -118.2588, 1, 624, 102)
(180100526, 2018, 1, 'January', '21-30', 'Hispanic', 'Female', 34.0487, -118.2588, 1, 624, 102)
(180100529, 2018, 1, 'January', '41-50', 'White', 'Male', 34.0713, -118.2291, 1, 740, 101)


In [3]:
# Query data from crime type table
crime_type_data = conn.execute("SELECT * FROM crime_type")

i = 0
for record in crime_type_data:
    print(record)
    i += 1
    if i == 10:
        break

(121, 'Sexual Assualt or Rape')
(122, 'Sexual Assualt or Rape')
(210, 'Theft, Robbery, or Burglary')
(220, 'Theft, Robbery, or Burglary')
(230, 'Assault')
(231, 'Assault')
(235, 'Child Maltreatment')
(236, 'Domestic Violence')
(237, 'Child Maltreatment')
(310, 'Theft, Robbery, or Burglary')


In [4]:
# Query data from crime table
area_data = conn.execute("SELECT * FROM area")

i = 0
for record in area_data:
    print(record)
    i += 1
    if i == 10:
        break

(1, 'Central')
(2, 'Rampart')
(3, 'Southwest')
(4, 'Hollenbeck')
(5, 'Harbor')
(6, 'Hollywood')
(7, 'Wilshire')
(8, 'West LA')
(9, 'Van Nuys')
(10, 'West Valley')


In [5]:
# Query data from crime table
premise_data = conn.execute("SELECT * FROM premise")

i = 0
for record in premise_data:
    print(record)
    i += 1
    if i == 10:
        break

(101, 'Freeway/Street')
(102, 'Sidewalk')
(103, 'Freeway/Street')
(104, 'Driveway')
(108, 'Parking Lot/Garage')
(109, 'Park')
(110, 'Freeway/Street')
(117, 'Beach')
(119, 'Residence')
(121, 'Yard')


## Joining the Tables

In [6]:
# creating database session
metadata = MetaData(bind=engine)
Base = automap_base(metadata=metadata)
# Use the Base class to reflect the database tables
Base.prepare(engine=engine, reflect=True)
Base.classes.keys()

['area', 'crime', 'crime_type', 'premise', 'la_crime']

In [7]:
# Collect the names of tables within the database
inspector = inspect(engine)
inspector.get_table_names()

['area', 'crime', 'crime_type', 'la_crime', 'premise']

In [8]:
# Using the inspector to print the column names within the 'crime' table and its types
columns = inspector.get_columns('crime')
for column in columns:
    print(column["name"], column["type"])

incident_id INTEGER
year INTEGER
month INTEGER
month_name VARCHAR
victim_age VARCHAR
victim_ethnicity VARCHAR
victim_gender VARCHAR
lat FLOAT
lon FLOAT
area_code INTEGER
crime_code INTEGER
premise_code INTEGER


In [9]:
# Reflect the database tables
metadata =  MetaData(bind=engine)
metadata.reflect()
crime_table = metadata.tables['crime']
area_table = metadata.tables['area']
crime_type_table = metadata.tables['crime_type']
premise_table = metadata.tables['premise']
crime_table

Table('crime', MetaData(bind=Engine(sqlite:///Resources/la_crime.db)), Column('incident_id', INTEGER(), table=<crime>, primary_key=True, nullable=False), Column('year', INTEGER(), table=<crime>), Column('month', INTEGER(), table=<crime>), Column('month_name', VARCHAR(), table=<crime>), Column('victim_age', VARCHAR(), table=<crime>), Column('victim_ethnicity', VARCHAR(), table=<crime>), Column('victim_gender', VARCHAR(), table=<crime>), Column('lat', FLOAT(), table=<crime>), Column('lon', FLOAT(), table=<crime>), Column('area_code', INTEGER(), ForeignKey('area.area_code'), table=<crime>), Column('crime_code', INTEGER(), ForeignKey('crime_type.crime_code'), table=<crime>), Column('premise_code', INTEGER(), ForeignKey('premise.premise_code'), table=<crime>), schema=None)

In [10]:
# Select the crime table
crime_type_tb = engine.execute(select([crime_type_table]))

# Get all the rows
rows = crime_type_tb.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

49
(121, 'Sexual Assualt or Rape')


In [11]:
# Select the crime type table
crime_tb = engine.execute(select([crime_table]))

# Get all the rows
rows = crime_tb.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

744444
(10304468, 2020, 1, 'January', '31-40', 'Black', 'Female', 34.0141, -118.2978, 3, 624, 501)


In [12]:
# Select the area table
area_tb = engine.execute(select([area_table]))

# Get all the rows
rows = area_tb.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

21
(1, 'Central')


In [13]:
# Select the premise table
premise = engine.execute(select([premise_table]))

# Get all the rows
rows = premise.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

93
(101, 'Freeway/Street')


In [14]:
# Define the SQL query
sql = text("""SELECT * 
            FROM crime
            LEFT JOIN area ON crime.area_code = area.area_code
            LEFT JOIN premise ON crime.premise_code = premise.premise_code
            LEFT JOIN crime_type ON crime.crime_code = crime_type.crime_code""")

# Execute the SQL query
la_crime = engine.execute(sql)

In [15]:
# Print the column names
print(f"Columns: {la_crime.keys()}")

Columns: ['incident_id', 'year', 'month', 'month_name', 'victim_age', 'victim_ethnicity', 'victim_gender', 'lat', 'lon', 'area_code', 'crime_code', 'premise_code', 'area_code', 'area_name', 'premise_code', 'premise', 'crime_code', 'crime']


In [16]:
# Get all the rows
rows = la_crime.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

744444
(10304468, 2020, 1, 'January', '31-40', 'Black', 'Female', 34.0141, -118.2978, 3, 624, 501, 3, 'Southwest', 501, 'Residence', 624, 'Assault')


In [17]:
# print first 10 rows
i = 0
for row in rows:
    print(row)
    if i == 10:
        break
    i += 1

(10304468, 2020, 1, 'January', '31-40', 'Black', 'Female', 34.0141, -118.2978, 3, 624, 501, 3, 'Southwest', 501, 'Residence', 624, 'Assault')
(20305364, 2019, 1, 'January', '41-50', 'Hispanic', 'Male', 34.0055, -118.2915, 3, 626, 502, 3, 'Southwest', 502, 'Residence', 626, 'Domestic Violence')
(170701073, 2018, 1, 'January', '11-20', 'Black', 'Male', 34.0421, -118.3456, 7, 888, 501, 7, 'Wilshire', 501, 'Residence', 888, 'Trespassing')
(180100001, 2018, 9, 'September', '31-40', 'White', 'Female', 34.0382, -118.2889, 20, 510, 101, 20, 'Olympic', 101, 'Freeway/Street', 510, 'Vehicle Theft')
(180100513, 2018, 1, 'January', '51-60', 'Black', 'Male', 34.0328, -118.2647, 1, 624, 101, 1, 'Central', 101, 'Freeway/Street', 624, 'Assault')
(180100514, 2018, 1, 'January', '31-40', 'White', 'Female', 34.0428, -118.2532, 1, 230, 108, 1, 'Central', 108, 'Parking Lot/Garage', 230, 'Assault')
(180100516, 2018, 1, 'January', '11-20', 'Other', 'Male', 34.0454, -118.2422, 1, 740, 101, 1, 'Central', 101, '

## Creating the final table for running queries 

In [8]:
from sqlalchemy.orm import sessionmaker
# creating database session
Session = sessionmaker(bind=engine)
session = Session()
metadata = MetaData(bind=engine)
Base = automap_base(metadata=metadata)
# Use the Base class to reflect the database tables
Base.prepare(engine = engine, reflect = True)
Base.classes.keys()

['area', 'crime', 'crime_type', 'premise', 'la_crime', 'la_crime2']

In [3]:
# Define the la_crime class , which will have a one-to-one relationship with the Area, premise and crime type tables
class CrimeJoined2(Base):
    __tablename__ = 'la_crime2'
    id = Column(Integer, primary_key=True, autoincrement=True)
    incident_id = Column(Integer)
    year = Column(Integer)
    month = Column(Integer)
    month_name = Column(String)
    victim_age = Column(String)
    victim_ethnicity = Column(String)
    victim_gender = Column(String)
    lat = Column(Float)
    lon = Column(Float)
    area_code = Column(Integer)
    crime_code = Column(Integer)
    premise_code = Column(Integer)
    area_name = Column(String, nullable=False)
    crime = Column(String, nullable=False)
    premise = Column(String, nullable=False)

In [4]:
# create the table in the database
metadata.create_all(bind=engine, checkfirst=True)

In [9]:
# Define the SQL query
sql = text("""SELECT * 
            FROM crime
            LEFT JOIN area ON crime.area_code = area.area_code
            LEFT JOIN premise ON crime.premise_code = premise.premise_code
            LEFT JOIN crime_type ON crime.crime_code = crime_type.crime_code""")
# Execute the SQL query
la_crime = engine.execute(sql)
rows = la_crime.fetchall()
print(len(rows))

# loop over the result set and insert each row into the new table
for row in rows:
    print(row)
    new_row = CrimeJoined2(incident_id = row.incident_id, 
                       year = row.year,
                       month = row.month,
                       month_name = row.month_name,
                       victim_age = row.victim_age, 
                       victim_ethnicity = row.victim_ethnicity,
                       victim_gender = row. victim_gender,
                       lat = row.lat,
                       lon = row.lon,
                       area_code = row.area_code,
                       crime_code = row.crime_code, 
                       premise_code= row.premise_code,
                       area_name = row.area_name, 
                       crime = row.crime,
                       premise = row.premise)
    session.add(new_row)

# commit the changes
session.commit()

744444
(10304468, 2020, 1, 'January', '31-40', 'Black', 'Female', 34.0141, -118.2978, 3, 624, 501, 3, 'Southwest', 501, 'Residence', 624, 'Assault')


UnmappedInstanceError: Class '__main__.CrimeJoined2' is not mapped

In [ ]:
inspector.get_table_names()

In [ ]:
# Select the la_crime table
la_crime = engine.execute(select([metadata.tables['la_crime_joined']]))

In [ ]:
# Get all the rows
rows = la_crime_tb.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

## Data Queries - using SQLAlchamey (copy to flask app to create functions)

use the table named la_crime for all queries (class is CrimeJoined)

In [11]:
# creating database session
metadata = MetaData(bind=engine)
Base = automap_base(metadata=metadata)
# Use the Base class to reflect the database tables
Base.prepare(engine = engine, reflect = True)
Base.classes.keys()

['area', 'crime', 'crime_type', 'premise', 'la_crime']

In [12]:
la_crime = engine.execute(select([metadata.tables['la_crime']]))

In [13]:
# Data for overall line chart by year - number of total crimes
query = text("""
    SELECT count('crime'), year 
    FROM la_crime
    GROUP BY year
    """)
result = engine.execute(query)
rows = result.fetchall()
crime_dict = []
for row in rows:
    print(row)
    crime_dict.append({"Year": row[1], "Total Crimes": row[0]})
print(crime_dict)

(164523, 2018)
(155471, 2019)
(132425, 2020)
(138336, 2021)
(153689, 2022)
[{'Year': 2018, 'Total Crimes': 164523}, {'Year': 2019, 'Total Crimes': 155471}, {'Year': 2020, 'Total Crimes': 132425}, {'Year': 2021, 'Total Crimes': 138336}, {'Year': 2022, 'Total Crimes': 153689}]


In [14]:
# Data for line chart by month - for specific year and crime
query = text("""
    SELECT count('crime'), year, month, month_name 
    FROM la_crime
    WHERE year = 2018 
    AND crime = 'Assault'
    GROUP BY month_name, month
    ORDER BY month    
    """)
result = engine.execute(query)
rows = result.fetchall()
age_dict = []
for row in rows:
    print(row)
    age_dict.append({"Month": row[3], "Total Crimes": row[0]})
print(crime_dict)

(2287, 2018, 1, 'January')
(2011, 2018, 2, 'February')
(2394, 2018, 3, 'March')
(2445, 2018, 4, 'April')
(2595, 2018, 5, 'May')
(2545, 2018, 6, 'June')
(2656, 2018, 7, 'July')
(2484, 2018, 8, 'August')
(2442, 2018, 9, 'September')
(2548, 2018, 10, 'October')
(2240, 2018, 11, 'November')
(2293, 2018, 12, 'December')
[{'Month': 'January', 'Total Crimes': 2287}, {'Month': 'February', 'Total Crimes': 2011}, {'Month': 'March', 'Total Crimes': 2394}, {'Month': 'April', 'Total Crimes': 2445}, {'Month': 'May', 'Total Crimes': 2595}, {'Month': 'June', 'Total Crimes': 2545}, {'Month': 'July', 'Total Crimes': 2656}, {'Month': 'August', 'Total Crimes': 2484}, {'Month': 'September', 'Total Crimes': 2442}, {'Month': 'October', 'Total Crimes': 2548}, {'Month': 'November', 'Total Crimes': 2240}, {'Month': 'December', 'Total Crimes': 2293}]


In [52]:
# Data for victim age by year and crime
query_age = text("""
    SELECT crime, year, Victim_age, count(*) AS total_count 
    FROM la_crime
    WHERE year = 2018 
    AND crime = 'Burglary from Vehicle'
    GROUP BY Victim_age
    """)

result = engine.execute(query_age)
rows = result.fetchall()
age_dict = []
total_count = 0
for row in rows:
    print(row)
    age_dict.append({"Age": row[2], "Age Count": row[3]})
    total_count += row[3]
print(age_dict, total_count)
for d in age_dict:
    d["Total People"] = total_count
    d["Age %"] = round((d["Age Count"]/d["Total People"]) * 100)
print(age_dict)

('Burglary from Vehicle', 2018, '1-10', 8)
('Burglary from Vehicle', 2018, '11-20', 780)
('Burglary from Vehicle', 2018, '21-30', 7985)
('Burglary from Vehicle', 2018, '31-40', 7555)
('Burglary from Vehicle', 2018, '41-50', 5304)
('Burglary from Vehicle', 2018, '51-60', 3746)
('Burglary from Vehicle', 2018, '61-70', 1874)
('Burglary from Vehicle', 2018, '71+', 626)
[{'Age': '1-10', 'Age Count': 8}, {'Age': '11-20', 'Age Count': 780}, {'Age': '21-30', 'Age Count': 7985}, {'Age': '31-40', 'Age Count': 7555}, {'Age': '41-50', 'Age Count': 5304}, {'Age': '51-60', 'Age Count': 3746}, {'Age': '61-70', 'Age Count': 1874}, {'Age': '71+', 'Age Count': 626}] 27878
[{'Age': '1-10', 'Age Count': 8, 'Total People': 27878, 'Age %': 0}, {'Age': '11-20', 'Age Count': 780, 'Total People': 27878, 'Age %': 3}, {'Age': '21-30', 'Age Count': 7985, 'Total People': 27878, 'Age %': 29}, {'Age': '31-40', 'Age Count': 7555, 'Total People': 27878, 'Age %': 27}, {'Age': '41-50', 'Age Count': 5304, 'Total People':

In [53]:
# Data for victim gender by year and crime
query_gender = text("""
    SELECT crime, year, Victim_gender, count(*) AS total_count
    FROM la_crime
    WHERE year = 2018 
    AND crime = 'Burglary from Vehicle'
    GROUP BY Victim_gender
    """)
result = engine.execute(query_gender)
rows = result.fetchall()
gender_dict = []
total_count = 0
for row in rows:
    print(row)
    gender_dict.append({"Gender": row[2], "Gender Count": row[3]})
    total_count += row[3]
print(gender_dict, total_count)
for d in gender_dict:
    d["Total People"] = total_count
    d["Gender %"] = round((d["Gender Count"]/d["Total People"]) * 100)
print(gender_dict)

('Burglary from Vehicle', 2018, 'Female', 12143)
('Burglary from Vehicle', 2018, 'Male', 15735)
[{'Gender': 'Female', 'Gender Count': 12143}, {'Gender': 'Male', 'Gender Count': 15735}] 27878
[{'Gender': 'Female', 'Gender Count': 12143, 'Total People': 27878, 'Gender %': 44}, {'Gender': 'Male', 'Gender Count': 15735, 'Total People': 27878, 'Gender %': 56}]


In [54]:
# Data for victim ethnicity by year and crime
query_ethnicity = text("""
    SELECT crime, year, Victim_ethnicity, count(*) AS total_count
    FROM la_crime
    WHERE year = 2018 
    AND crime = 'Burglary from Vehicle'
    GROUP BY Victim_ethnicity
    """)
result = engine.execute(query_ethnicity)
rows = result.fetchall()
ethnicity_dict = []
total_count = 0
for row in rows:
    print(row)
    ethnicity_dict.append({"Ethnicity": row[2], "Ethnicity Count": row[3]})
    total_count += row[3]
print(ethnicity_dict, total_count)
for d in ethnicity_dict:
    d["Total People"] = total_count
    d["Ethnicity %"] = round((d["Ethnicity Count"]/d["Total People"]) * 100)
print(ethnicity_dict)

('Burglary from Vehicle', 2018, 'Asian', 1481)
('Burglary from Vehicle', 2018, 'Asian Indian', 4)
('Burglary from Vehicle', 2018, 'Black', 3588)
('Burglary from Vehicle', 2018, 'Hispanic', 10356)
('Burglary from Vehicle', 2018, 'Native American', 7)
('Burglary from Vehicle', 2018, 'Other', 3473)
('Burglary from Vehicle', 2018, 'Pacific Islander/Hawaiian', 7)
('Burglary from Vehicle', 2018, 'White', 8962)
[{'Ethnicity': 'Asian', 'Ethnicity Count': 1481}, {'Ethnicity': 'Asian Indian', 'Ethnicity Count': 4}, {'Ethnicity': 'Black', 'Ethnicity Count': 3588}, {'Ethnicity': 'Hispanic', 'Ethnicity Count': 10356}, {'Ethnicity': 'Native American', 'Ethnicity Count': 7}, {'Ethnicity': 'Other', 'Ethnicity Count': 3473}, {'Ethnicity': 'Pacific Islander/Hawaiian', 'Ethnicity Count': 7}, {'Ethnicity': 'White', 'Ethnicity Count': 8962}] 27878
[{'Ethnicity': 'Asian', 'Ethnicity Count': 1481, 'Total People': 27878, 'Ethnicity %': 5}, {'Ethnicity': 'Asian Indian', 'Ethnicity Count': 4, 'Total People': 27

In [50]:
crime_dict = {"Age Data": age_dict, "Gender Data": gender_dict, "Ethnicity Data": ethnicity_dict}
crime_dict

{'Age Data': [{'Age': '1-10', 'Age Count': 8},
  {'Age': '11-20', 'Age Count': 780},
  {'Age': '21-30', 'Age Count': 7985},
  {'Age': '31-40', 'Age Count': 7555},
  {'Age': '41-50', 'Age Count': 5304},
  {'Age': '51-60', 'Age Count': 3746},
  {'Age': '61-70', 'Age Count': 1874},
  {'Age': '71+', 'Age Count': 626}],
 'Gender Data': [{'Gender': 'Female',
   'Gender Count': 12143,
   'Total People': 27878,
   'Gender %': 44},
  {'Gender': 'Male',
   'Gender Count': 15735,
   'Total People': 27878,
   'Gender %': 56}],
 'Ethnicity Data': [{'Ethnicity': 'Asian',
   'Ethnicity Count': 1481,
   'Total People': 27878,
   'Ethnicity %': 5},
  {'Ethnicity': 'Asian Indian',
   'Ethnicity Count': 4,
   'Total People': 27878,
   'Ethnicity %': 0},
  {'Ethnicity': 'Black',
   'Ethnicity Count': 3588,
   'Total People': 27878,
   'Ethnicity %': 13},
  {'Ethnicity': 'Hispanic',
   'Ethnicity Count': 10356,
   'Total People': 27878,
   'Ethnicity %': 37},
  {'Ethnicity': 'Native American',
   'Ethnicit

In [58]:
# Data for map by year and crime
query_map = text("""
    SELECT crime, year, lat, lon, area_name, premise, count(*) AS total_count
    FROM la_crime
    WHERE year = 2018 
    AND crime = 'Burglary from Vehicle'
    GROUP BY lat, lon, area_name, premise
    LIMIT 100
    """)

result = engine.execute(query_map)
rows = result.fetchall()
map_dict = []
print(len(rows))
for row in rows:
    print(row)
    map_dict.append({"lat": row[2], "lon": row[3], "Area": row[4],"Premise": row[5], "Number of Crimes": row[6]})
print(map_dict)

100
('Burglary from Vehicle', 2018, 33.7065, -118.2928, 'Harbor', 'Parking Lot/Garage', 1)
('Burglary from Vehicle', 2018, 33.707, -118.2939, 'Harbor', 'Freeway/Street', 1)
('Burglary from Vehicle', 2018, 33.7079, -118.2879, 'Harbor', 'Driveway', 1)
('Burglary from Vehicle', 2018, 33.7087, -118.2862, 'Harbor', 'Residence', 1)
('Burglary from Vehicle', 2018, 33.7089, -118.2855, 'Harbor', 'Freeway/Street', 1)
('Burglary from Vehicle', 2018, 33.7096, -118.2906, 'Harbor', 'Driveway', 1)
('Burglary from Vehicle', 2018, 33.7096, -118.2862, 'Harbor', 'Driveway', 1)
('Burglary from Vehicle', 2018, 33.7105, -118.286, 'Harbor', 'Parking Lot/Garage', 1)
('Burglary from Vehicle', 2018, 33.7105, -118.286, 'Harbor', 'Residence', 1)
('Burglary from Vehicle', 2018, 33.7115, -118.2879, 'Harbor', 'Freeway/Street', 1)
('Burglary from Vehicle', 2018, 33.7115, -118.2879, 'Harbor', 'Residence', 1)
('Burglary from Vehicle', 2018, 33.7116, -118.2981, 'Harbor', 'Freeway/Street', 1)
('Burglary from Vehicle', 20